In [1]:
import ray
ray.init()

2023-08-04 17:14:46,243	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.17
Ray version:,2.6.1
Dashboard:,http://127.0.0.1:8265


In [2]:
resources = ray.cluster_resources()
print(resources)

{'CPU': 32.0, 'node:158.125.233.246': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 34785067008.0, 'GPU': 1.0, 'accelerator_type:G': 1.0, 'memory': 71165156352.0}


In [3]:
import gymnasium as gym
import numpy as np

class SeededFetchReachEnv(gym.Wrapper):
    def __init__(self, env, seed=None):
        super().__init__(env)
        self.seed = seed

    def reset(self, seed=None, options=None):
        obs, info = self.env.reset(seed=self.seed)
        return obs, info

    def seed(self, seed=None):
        self.np_random, seed = np.random.SeedSequence(seed)
        return [seed]

# Usage:
#env = gym.make('FetchPickAndPlace-v2')
#wrapped_env = SeededFetchPickAndPlaceEnv(env, 959)


In [4]:
#env = gym.make('FrankaPickPlaceFixed-v0')

In [5]:
def env_creator(env_config={}):
    import gymnasium as gym
    env = gym.make('FetchReach-v2', render_mode='rgb_array', reward_type='sparse', max_episode_steps=500)
    env = SeededFetchReachEnv(env, 960)
    env.reset()
    return env

In [6]:
from ray.tune.registry import register_env
register_env("Gymnasium_Reach_0_Sparse_Single_Task-500_Steps", env_creator)

In [7]:
# Serialize with cloudpickle and save to a file
#with open("Gymnasium_Pick_Place_0.pkl", "wb") as f:
   # cloudpickle.dump(env_creator, f)

In [8]:
from ray import tune

In [9]:
tune.run("PPO",
             # algorithm specific configuration
             config={"env": "Gymnasium_Reach_0_Sparse_Single_Task-500_Steps",  #
                     "framework": "torch",
                     #"seed":958,
                     "num_gpus": 1,
                     "num_rollout_workers": 24,
                     #"num_envs_per_worker": 1,
                     #"num_gpus_per_worker": 0,
                     "train_batch_size": 2048,
                     #"dreamer_train_iters": 8,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 32,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.00015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "use_lstm": True,
                     "use_attention": True,
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2,
                     "evaluation_parallel_to_training": True,
                     "evaluation_num_workers": 1,
                     "disable_env_checking":True
                     },
             local_dir="../R3L-LOGS",  # directory to save results
             checkpoint_freq=10,  # frequency between checkpoints
             #keep_checkpoints_num=10,
             stop={"training_iteration": 5000}
             )

2023-08-04 17:14:47,544	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/lunet/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-08-04 17:14:48,517	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-08-04 17:14:48,562	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-08-04 17:14:48,564	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-0

2023-08-04 17:14:48,623	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-04 17:14:48,623	WARNING algorithm_config.py:656 -- Cannot create PPOConfig from given `config_dict`! Property use_lstm not supported.
2023-08-04 17:14:48,623	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=273030) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=27

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,evaluation,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_Gymnasium_Reach_0_Sparse_Single_Task-500_Steps_08c42_00000,561152,"{'ObsPreprocessorConnector_ms': 0.02178812026977539, 'StateBufferConnector_ms': 0.004323244094848633, 'ViewRequirementAgentConnector_ms': 0.2604405879974365}","{'num_env_steps_sampled': 561152, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 561152, 'num_agent_steps_trained': 0}",{},500,{},-3,-7.71,-16,8,"{'sampler_results': {'episode_reward_max': -6.0, 'episode_reward_min': -7.0, 'episode_reward_mean': -6.5, 'episode_len_mean': 500.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-6.0, -7.0], 'episode_lengths': [500, 500]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.2690024382197921, 'mean_inference_ms': 0.6680290969470005, 'mean_action_processing_ms': 0.09800127731870607, 'mean_env_wait_ms': 1.1266035056309196, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.010776519775390625, 'StateBufferConnector_ms': 0.0021576881408691406, 'ViewRequirementAgentConnector_ms': 0.1353144645690918}}, 'episode_reward_max': -6.0, 'episode_reward_min': -7.0, 'episode_reward_mean': -6.5, 'episode_len_mean': 500.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-6.0, -7.0], 'episode_lengths': [500, 500]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.2690024382197921, 'mean_inference_ms': 0.6680290969470005, 'mean_action_processing_ms': 0.09800127731870607, 'mean_env_wait_ms': 1.1266035056309196, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.010776519775390625, 'StateBufferConnector_ms': 0.0021576881408691406, 'ViewRequirementAgentConnector_ms': 0.1353144645690918}, 'num_agent_steps_sampled_this_iter': 1000, 'num_env_steps_sampled_this_iter': 1000, 'timesteps_this_iter': 1000, 'num_healthy_workers': 1, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}","{'learner': {'__all__': {'num_agent_steps_trained': 32.0, 'num_env_steps_trained': 2048.0, 'total_loss': 0.2073877948569134}, 'default_policy': {'total_loss': 0.2073877948569134, 'policy_loss': -0.0007749216310912743, 'vf_loss': 0.208258129144808, 'vf_loss_unclipped': 0.6059660391889268, 'vf_explained_var': -9.504146873950958e-07, 'entropy': 0.6360982847108971, 'mean_kl_loss': 0.015225443094323055, 'gradients_default_optimizer_global_norm': 4.74554519134108, 'curr_lr': 0.00015, 'curr_entropy_coeff': 0.00015, 'curr_kl_coeff': 1.1700024963165435e-33}}, 'num_env_steps_sampled': 561152, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 561152, 'num_agent_steps_trained': 0}",561152,0,561152,2048,690.697,0,0,0,0,24,0,0,0,"{'cpu_util_percent': 23.5, 'ram_util_percent': 23.3}",{},{},{},"{'mean_raw_obs_processing_ms': 0.4895737414533825, 'mean_inference_ms': 1.228079507501159, 'mean_action_processing_ms': 0.16673688101701237, 'mean_env_wait_ms': 1.948183672781492, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -3.0, 'episode_reward_min': -16.0, 'episode_reward_mean': -7.71, 'episode_len_mean': 500.0, 'episode_media': {}, 'episodes_this_iter': 8, 'policy_reward_min': {}, 'policy

2023-08-04 17:26:20,107	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-08-04 17:26:21,742	INFO tune.py:1148 -- Total run time: 694.20 seconds (691.49 seconds for the tuning loop).
2023-08-04 17:26:21,743	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)


In [19]:
env = env_creator({})

In [20]:
import numpy as np

from ray.rllib.policy.policy import Policy     

# Use the `from_checkpoint` utility of the Policy class:
my_restored_policy = Policy.from_checkpoint("../R3L-LOGS/PPO/PPO_Gymnasium_Reach_0_Sparse_Single_Task-500_Steps_08c42_00000_0_2023-08-04_17-14-48/checkpoint_000230/policies/default_policy")

# Use the restored policy for serving actions.
obs,  reward, terminated, truncated, info =   env.step(env.action_space.sample())# individual CartPole observation
#action = my_restored_policy.compute_single_action(np.append(obs.get('observation'), obs.get('achieved_goal'), obs.get('desired_goal')))


2023-08-04 17:29:21,103	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


In [21]:
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
img = plt.imshow(env.render())
counter = 0
for i in range(0, 100):
    counter = counter +i
    action = my_restored_policy.compute_single_action(np.concatenate([obs.get('observation'), obs.get('desired_goal'), obs.get('achieved_goal')]))

    obs,  reward, terminated, truncated, info =   env.step(env.action_space.sample())# individual CartPole observation

    '''if counter >=50:
        env.reset()
        img.set_data(env.render()) # Just update the data
        ipythondisplay.display(plt.gcf())
        ipythondisplay.clear_output(wait=True)
        counter = 0'''

    #print(f"Computed action {action} from given CartPole observation.")
    img.set_data(env.render()) # Just update the data
    ipythondisplay.display(plt.gcf())
    ipythondisplay.clear_output(wait=True)